# ERT calibration and GF calibration
This notebook explores how ECa values acquired at 1 m using the F-1m calibration can be corrected using ERT calibration.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')
sys.path.append('../../pyr2/src')
from resipy import R2
from resipy.meshTools import vtk_import
from emagpy import Problem

datadir = '../src/examples/boxford-calib/'

In [12]:
# inverting one line
k1er = R2(datadir + 'wd')
k1er.createSurvey(datadir + 'wd/protocol.dat', ftype='ProtocolDC')
k1er.importElec(datadir + 'wd/elec.csv')
k1er.createMesh('trian', cl=0.5, fmd=5)
k1er.invert()
k1er.getResults()
fig, ax = plt.subplots(figsize=(10,2))
k1er.showResults(ax=ax)

In [17]:
%load_ext autoreload
%autoreload 2

In [2]:
# no correction
k1 = Problem()
k1.createSurvey(datadir + 'eca_raw.csv')

# with GF correction
k2 = Problem()
k2.createSurvey(datadir + 'eca_raw.csv')
k2.gfCorrection(calib='F-1m')

# like if it was collected at 0 m
k3 = Problem()
k3.createSurvey(datadir + 'eca_raw.csv')
k3.hx = np.zeros(6) # make it like data were collected at 0 m

# ERT calibrated but no GF corrected
k4 = Problem()
k4.createSurvey(datadir + 'eca_raw.csv')
k4.calibrate(datadir + 'eca_raw.csv',
             fnameEC=datadir + 'eri_ec.csv',
#              fnameResMod=datadir + 'wd/invdir/f001_res.dat',
             forwardModel='FSlin', apply=True)

# ERT calibrated and GF corrected
k5 = Problem()
k5.createSurvey(datadir + 'eca_raw.csv')
k5.gfCorrection(calib='F-1m')
k5.calibrate(datadir + 'eca_raw.csv',
             fnameEC=datadir + 'eri_ec.csv',
#              fnameResMod=datadir + 'wd/invdir/f001_res.dat',
             forwardModel='FSlin', calib='F-1m', apply=True)

ks = [k1, k2, k3, k4, k5]
for k in ks:
    k.setInit(depths0=np.linspace(0.01, 5, 10))
    k.invert(forwardModel='FSlin', method='Gauss-Newton')

In [14]:
emSens(np.linspace(0,3),s=1.48,coilPosition='vcp')

In [5]:
def emSens(depths,s,coilPosition, hx=0, rescaled=False):
    """return mcNeil senstivity values
    
    depths (positive and increasing with depth)
    s = distance between coils
    coilPosition is "hcp" for horizontal coplanar (HCP) and "vcp" for VCP
    x = range on which compute the sensitivity (array_like)
     """
    depths = depths + np.abs(hx)
    z=np.array(depths)/s
    # from mcNeil 1980 in Callegary2007
    if coilPosition == 'hcp':
        cs = 1/np.sqrt(4*z**2+1)
    if coilPosition == 'vcp':
        cs = np.sqrt(4*z**2+1)-2*z
    if coilPosition == 'prp':
        cs = 1-2*z/np.sqrt(4*z**2 + 1)
    
    if (hx != 0) and rescaled is True: # rescale the cumulative sensitivity so that it reaches 1
        cs = cs/cs[0]

    return cs # all but the first one as it is just use to scale everything

In [47]:
# figure
fig, axs = plt.subplots(len(ks), 2, figsize=(10,11))
letters = ['a','b','c','d','e','f','g','h','i','j']
labs = ['no correction', 'GF correction', 'like if at 0 m', 'ERT calibrated', 'ERT calib + GF corr']
for i, k in enumerate(ks):
    k.show(ax=axs[i,0], vmin=0, vmax=50)
    axs[i,0].get_legend().remove()
    axs[i,0].set_title('({:s}) Apparent ({:s})'.format(letters[i*2], labs[i]))
    k.showResults(ax=axs[i,1], vmin=0, vmax=50)
    axs[i,1].set_title('({:s}) Inverted ({:s})'.format(letters[i*2+1], labs[i]))

In [48]:
# comparing with ERT
vmin, vmax = 0, 50
fig, axs = plt.subplots(6, 1, figsize=(8,8), sharex=True, sharey=False)
ax = axs[0]
k1.showResults(ax=ax, vmin=vmin, vmax=vmax)
ax.set_title('(a) EMI no correction')
ax.set_xlabel('')
ax = axs[1]
k2.showResults(ax=ax, vmin=vmin, vmax=vmax)
ax.set_title('(b) EMI with GF correction')
ax.set_xlabel('')
ax = axs[2]
k3.showResults(ax=ax, vmin=vmin, vmax=vmax)
ax.set_title('(c) EMI like at 0 m')
ax.set_xlabel('')
ax = axs[3]
k4.showResults(ax=ax, vmin=vmin, vmax=vmax)
ax.set_title('(d) EMI with ERT calib')
ax.set_xlabel('')
ax = axs[4]
k5.showResults(ax=ax, vmin=vmin, vmax=vmax)
ax.set_title('(e) EMI with ERT calib + GF correction')
ax.set_xlabel('')
ax = axs[5]
k1er.showResults(ax=ax, attr='Conductivity(mS/m)', vmin=vmin, vmax=vmax,
                 sens=False, color_map='viridis_r')
ax.set_title('(f) ERT transect')

# adjust the x axis of subplot 1 to match the others
pos = axs[-1].get_position()
cbar0 = fig.axes[-1].get_position()
for i, ax in enumerate(axs[:-1]):
    pos0 = ax.get_position()
    ax.set_position([pos0.x0, pos0.y0, pos.width, pos.height])
    cbar = fig.axes[len(axs) + i]
    pos1 = cbar.get_position()
    cbar.set_position([cbar0.x0, pos1.y0, pos1.width, pos1.height])

for ax in axs:
    ax.set_aspect('auto')

In [49]:
for k in ks:
    k.showOne2one()

## Conclusions
- Applying the ERT calibration alone is sufficient to account for GF calibration as well
- Applying the GF correction and then the ERT calibration leads to the same results as just applying the ERT calibration alone
- Hence, if ERT data are available, we recommend to just apply the ERT correction alone and not the GF calibration.